##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)

In [5]:
dsf = xr.Dataset()
for cc in range(13):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

In [6]:
USER = os.environ['USER']
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [7]:
%%time
dsf.load()

CPU times: user 5.56 ms, sys: 0 ns, total: 5.56 ms
Wall time: 5.57 ms


<xarray.Dataset>
Dimensions:          (nlat: 384, nlon: 320, year: 70)
Coordinates:
    TLAT             (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    TLONG            (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    ULAT             (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.41 72.41
    ULONG            (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
  * year             (year) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_0_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_0-200m     (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_200-600m   (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    ...               ...
    Phi_10_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_10_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [8]:
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_yearly.nc', mode='w')

## LME monthly to yearly

In [9]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:          (lme: 65, time: 840)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 ... 58 59 60 61 62 63 64 65
  * time             (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Data variables: (12/26)
    Phi_0_0-200m     (lme, time) float64 ...
    Phi_0_200-600m   (lme, time) float64 ...
    Phi_1_0-200m     (lme, time) float64 ...
    Phi_1_200-600m   (lme, time) float64 ...
    Phi_2_0-200m     (lme, time) float64 ...
    Phi_2_200-600m   (lme, time) float64 ...
    ...               ...
    Phi_10_0-200m    (lme, time) float64 ...
    Phi_10_200-600m  (lme, time) float64 ...
    Phi_11_0-200m    (lme, time) float64 ...
    Phi_11_200-600m  (lme, time) float64 ...
    Phi_12_0-200m    (lme, time) float64 ...
    Phi_12_200-600m  (lme, time) float64 ...

In [10]:
dsf = xr.Dataset()
for cc in range(13):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

In [11]:
USER = os.environ['USER']
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)

In [12]:
%%time
dsf.load()

CPU times: user 1.06 ms, sys: 50 µs, total: 1.11 ms
Wall time: 1.12 ms


<xarray.Dataset>
Dimensions:          (lme: 65, year: 70)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 ... 58 59 60 61 62 63 64 65
  * year             (year) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Data variables: (12/26)
    Phi_0_0-200m     (lme, year) float64 0.603 0.5966 0.6066 ... 0.5914 0.6149
    Phi_0_200-600m   (lme, year) float64 0.1243 0.12 0.1229 ... 0.08208 0.08221
    Phi_1_0-200m     (lme, year) float64 0.7092 0.7011 0.71 ... 0.6796 0.706
    Phi_1_200-600m   (lme, year) float64 0.1445 0.1394 0.1425 ... 0.0949 0.09502
    Phi_2_0-200m     (lme, year) float64 0.8353 0.825 0.8322 ... 0.7815 0.8111
    Phi_2_200-600m   (lme, year) float64 0.1679 0.1619 0.1652 ... 0.1097 0.1098
    ...               ...
    Phi_10_0-200m    (lme, year) float64 3.256 3.192 3.109 ... 2.513 2.448 2.511
    Phi_10_200-600m  (lme, year) float64 0.5623 0.5408 0.5409 ... 0.3506 0.35
    Phi_11_0-200m    (lme, year) float64 3.883 3.803 3.687 ... 2.903 2.831 2.899
    Phi_11_200-600m  (lme, year) float64 0.6546 0.6293 0.6277 ... 0.4054 0.4046
    Phi_12_0-200m    (lme, year) float64 4.636 4.536 4.377 ... 3.354 3.276 3.347
    Phi_12_200-600m  (lme, year) float64 0.7621 0.7324 0.7285 ... 0.4689 0.4677

In [13]:
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_yearly.nc', mode='w')